In [15]:
import os
import os.path
import PIL
from PIL import Image
import cv2
import numpy as np

In [41]:
DIR = 'output/train/happy/' #directory
glass_img = cv2.imread('glass_mask.png')
# SAVE_DIR = 'output/' #output directory

In [42]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [44]:
for file in os.listdir(DIR):
    image = cv2.imread(os.path.join(DIR,file))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    centers = []
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # iterating over the face detected
    for (x, y, w, h) in faces:

        # create two Regions of Interest.
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = image[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Store the coordinates of eyes in the image to the 'center' array
        for (ex, ey, ew, eh) in eyes:
            centers.append((x + int(ex + 0.5 * ew), y + int(ey + 0.5 * eh)))

    if len(centers) > 1:
        # change the given value of 2.15 according to the size of the detected face
        print(centers)
        glasses_width = 2.15 * abs(centers[1][0] - centers[0][0])
        overlay_img = np.ones(image.shape, np.uint8) * 255
        h, w = glass_img.shape[:2]
        scaling_factor = glasses_width / w

        overlay_glasses = cv2.resize(glass_img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

        x = centers[0][0] if centers[0][0] < centers[1][0] else centers[1][0]

        # The x and y variables below depend upon the size of the detected face.
        x -= 0.25 * overlay_glasses.shape[1]
        y += 1.1 * overlay_glasses.shape[0]

        # Slice the height, width of the overlay image.
        h, w = overlay_glasses.shape[:2]
        overlay_img[int(y):int(y + h), int(x):int(x + w)] = overlay_glasses

        # Create a mask and generate it's inverse.
        gray_glasses = cv2.cvtColor(overlay_img, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(gray_glasses, 110, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)
        temp = cv2.bitwise_and(image, image, mask=mask)

        temp2 = cv2.bitwise_and(overlay_img, overlay_img, mask=mask_inv)
        final_img = cv2.add(temp, temp2)
        
        #save the new images in new dir
        cv2.imwrite(os.path.join(DIR,file), final_img)
        centers.clear()
        
    
    

[(342, 386), (204, 380), (221, 524), (337, 515)]
[(206, 382), (225, 530), (346, 388)]
[(192, 384), (358, 383)]
[(211, 387), (344, 382)]
[(353, 386), (219, 533), (198, 383)]
[(222, 385), (336, 382), (234, 524)]
[(349, 380), (218, 526), (197, 380), (335, 527)]
[(207, 382), (227, 534), (253, 470), (341, 381)]
[(343, 381), (201, 380), (338, 526), (217, 526), (254, 483)]
[(342, 384), (211, 385), (291, 539)]
[(330, 385), (224, 523), (208, 381), (352, 383), (235, 380)]
[(345, 380), (207, 383), (247, 464), (274, 500)]
[(200, 380), (219, 530), (339, 380)]
[(353, 514), (224, 518), (235, 520), (343, 376), (206, 388)]
[(214, 380), (226, 516), (334, 522), (339, 384)]
[(347, 380), (201, 385), (244, 525), (225, 525), (331, 524)]
[(196, 383), (357, 383), (217, 544), (338, 542)]
[(345, 385), (207, 384), (227, 521), (342, 520)]
[(207, 375), (213, 528), (348, 382)]
[(251, 471), (205, 378), (221, 518), (341, 382), (331, 520)]
[(200, 381), (354, 379), (220, 534)]
[(352, 380), (202, 381), (249, 479)]
[(234,

ValueError: could not broadcast input array from shape (181,520,3) into shape (181,485,3)